# ModOAP - Classification automatique de textes

Ce script permet la classification automatique de blocs de texte issus de documents Gallica, à partir de modèles de classification préalablement entraînés.

Les modèles entraînés doivent être importés : ils peuvent être obtenus via le script ModOAP - Classification automatique de textes (Entrainement)

Trois types de classifications possibles :

1.   Classification binaire : associer un texte à une étiquette ou 0 (une seule étiquette possible)
2.   Classification multiclass : associer un texte à une étiquette parmi un jeu d'étiquettes
3.   Classification multilabel : associer un texte à une ou plusieurs étiquettes parmi un jeu d'étiquettes



**Ce script doit être lancé dans un environnement d'exécution GPU : Execution -> Modifier le type d'exécution -> GPU**

In [ ]:
#@markdown ### Synchronisation d'un compte Google Drive et installation des pré-requis (Nécessaire)
#@markdown Lancer la cellule et se rendre sur l'url générée par Google pour connecter un compte Drive si demandé.

#@markdown Cette cellule doit redémarrer l'environnement d'exécution.

# chargement d'un google drive
import os
from google.colab import drive

if not os.path.exists("/content/drive/MyDrive/") :
  drive.mount('/content/drive/')

#Importation de SimpleTransformers
try:
  from simpletransformers.classification import ClassificationModel, ClassificationArgs, MultiLabelClassificationModel, MultiLabelClassificationArgs

except ImportError:
  !pip install -q --no-color --upgrade transformers
  !pip install -q --no-color simpletransformers
  #!pip install --upgrade transformers
  #!pip install simpletransformers
  import os
  os.kill(os.getpid(), 9)


# Classification Binaire



In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import os 
import json


#@markdown ### Importation du modèle Binaire entraîné
#@markdown Entrer le chemin du dossier contenant le modèle entraîné :
chemin_modele = "" #@param {type:"string"}
#@markdown Exemple de chemin:
#@markdown /content/drive/MyDrive/Outils_Modoap/Classification_Texte/Classification_blocs_texte/modele_binaire/

model = ClassificationModel("camembert", chemin_modele)

#@markdown ### Importation du fichier test à annoter automatiquement
#@markdown Entrer le chemin du fichier JSON de blocs texte non-annotés pour la classification :
fichier_json = "" #@param {type:"string"}
#@markdown Exemple de chemin:
#@markdown /content/drive/MyDrive/Datasets/Documents_jsons/bpt6k96903906_texte_structure.json


#@markdown ### Spécification du dossier de destination
#@markdown Entrer le chemin du dossier où sauvegarder les résultats :
dossier_resultats = "" #@param {type:"string"}
#@markdown Exemple de chemin:
#@markdown /content/drive/MyDrive/Outils_Modoap/Classification_Texte/Classification_blocs_texte/resultats_binaire/
if not os.path.exists(dossier_resultats) :
  os.makedirs(dossier_resultats)

# Classification
with open(fichier_json, "r") as f:
  dico = json.load(f)
dico2 = dico
test = [dico["Text_Blocks"][x]["Content"] for x in dico["Text_Blocks"].keys()]

test_predictions, raw_outputs = model.predict(test)

# Sauvegarde vers un csv de résultats

dfres = pd.read_csv(os.path.join(chemin_modele,"dftemp.csv"), sep="\t")
classe = dfres.columns[-1]
dfres["text"] = test
dfres[classe] = test_predictions
dfres = dfres.fillna(0) # with 0s rather than NaNs

fich_result = os.path.join(dossier_resultats,str(fichier_json.split("/")[-1])[:-5]+"_classification_binaire.csv")
dfres.to_csv(fich_result, index=True, sep="\t")


# Sauvegarde vers un JSON avec blocs texte annotés
i = 0
for textblock_id in dico2["Text_Blocks"] :
  if str(test_predictions[i]) == "1":
    try :
      dico2["Text_Blocks"][textblock_id]["Etiquettes"] = dico2["Text_Blocks"][textblock_id]["Etiquettes"]+[classe]
    except KeyError :
      dico2["Text_Blocks"][textblock_id]["Etiquettes"] = [classe]
  else : pass
  i+=1

json_result = os.path.join(dossier_resultats,str(fichier_json.split("/")[-1])[:-5]+"_classification_binaire.json")
with open(json_result, "w") as fil:
  json.dump(dico2, fil)
print("Les résultats peuvent être visualisés dans le fichier JSON : ", "/".join(json_result.split("/")[-2:]))

print("Les résultats peuvent être visualisés dans le tableau : ", "/".join(fich_result.split("/")[-2:]))

dfres.head()


# Classification Multiclass 

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import os 
import json


#@markdown ### Importation du modèle MultiClass entraîné
#@markdown Entrer le chemin du dossier contenant le modèle entraîné :
chemin_modele = "" #@param {type:"string"}
#@markdown Exemple de chemin:
#@markdown /content/drive/MyDrive/Outils_Modoap/Classification_Texte/Classification_blocs_texte/modele_multiclass/

model = ClassificationModel("camembert", chemin_modele)

#@markdown ### Importation du fichier test à annoter automatiquement
#@markdown Entrer le chemin du fichier JSON de blocs texte non-annotés pour la classification :
fichier_json = "" #@param {type:"string"}
#@markdown Exemple de chemin:
#@markdown /content/drive/MyDrive/Datasets/Documents_jsons/bpt6k96903906_texte_structure.json

#@markdown ### Spécification du dossier de destination
#@markdown Entrer le chemin du dossier où sauvegarder les résultats :
dossier_resultats = "" #@param {type:"string"}
#@markdown Exemple de chemin:
#@markdown /content/drive/MyDrive/Outils_Modoap/Classification_Texte/Classification_blocs_texte/resultats_multiclass/
if not os.path.exists(dossier_resultats) :
  os.makedirs(dossier_resultats)

# Classification
with open(fichier_json, "r") as f:
  dico = json.load(f)
dico2 = dico
test = [dico["Text_Blocks"][x]["Content"] for x in dico["Text_Blocks"].keys()]

test_predictions, raw_outputs = model.predict(test)

# Sauvegarde vers un csv de résultats

with open(os.path.join(chemin_modele,"class_temp.json"), "r") as f:
  classtemp = json.load(f)

dfres= pd.DataFrame(columns=["text", "prediction"])
dfres["text"] = test
preds = []
for labl in test_predictions :
  for nom, id in classtemp.items() :
    if id == labl :
      preds.append(nom)
dfres["prediction"] = preds
dfres = dfres.fillna(0) # with 0s rather than NaNs

fich_result = os.path.join(dossier_resultats,str(fichier_json.split("/")[-1])[:-5]+"_classification_binaire.csv")
dfres.to_csv(fich_result, index=True, sep="\t")


# Sauvegarde vers un JSON avec blocs texte annotés
i = 0
for textblock_id in dico2["Text_Blocks"] :
  try :
    dico2["Text_Blocks"][textblock_id]["Etiquettes"] = dico2["Text_Blocks"][textblock_id]["Etiquettes"]+[preds[i]]
  except KeyError :
    dico2["Text_Blocks"][textblock_id]["Etiquettes"] = [preds[i]]
  i+=1

json_result = os.path.join(dossier_resultats,str(fichier_json.split("/")[-1])[:-5]+"_classification_binaire.json")
with open(json_result, "w") as fil:
  json.dump(dico2, fil)
print("Les résultats peuvent être visualisés dans le fichier JSON : ", "/".join(json_result.split("/")[-2:]))

print("Les résultats peuvent être visualisés dans le tableau : ", "/".join(fich_result.split("/")[-2:]))

dfres.head()


# Classification Multilabel

---



In [ ]:
import os 
import json
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from simpletransformers.classification import MultiLabelClassificationModel, MultiLabelClassificationArgs
import logging
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)



#@markdown ### Importation du modèle MultiLabel entraîné
#@markdown Entrer le chemin du dossier checkpoint à l'intérieur du dossier contenant le modèle entraîné :
chemin_modele = "" #@param {type:"string"}
#@markdown Exemple de chemin:
#@markdown /content/drive/MyDrive/Outils_Modoap/Classification_Texte/Classification_blocs_texte/modele_multilabel/checkpoint-53-epoch-1

model = MultiLabelClassificationModel("camembert", chemin_modele)

#@markdown ### Importation du fichier test à annoter automatiquement
#@markdown Entrer le chemin du fichier JSON de blocs texte non-annotés pour la classification :
fichier_json = "" #@param {type:"string"}
#@markdown Exemple de chemin:
#@markdown /content/drive/MyDrive/Datasets/Documents_jsons/bpt6k96903906_texte_structure.json


#@markdown ### Spécification du dossier de destination
#@markdown Entrer le chemin du dossier où sauvegarder les résultats :
dossier_resultats = "" #@param {type:"string"}
#@markdown Exemple de chemin:
#@markdown /content/drive/MyDrive/Outils_Modoap/Classification_Texte/Classification_blocs_texte/resultats_multilabel/
if not os.path.exists(dossier_resultats) :
  os.makedirs(dossier_resultats)

# Classification
with open(fichier_json, "r") as f:
  dico = json.load(f)
dico2 = dico
test = [dico["Text_Blocks"][x]["Content"] for x in dico["Text_Blocks"].keys()]

test_predictions, raw_outputs = model.predict(test)


##########################################################################
print(test_predictions[:10])
##########################################################################

# Sauvegarde vers un csv de résultats

dftemp = pd.read_csv(os.path.join("/".join(chemin_modele.split("/")[:-1]),"dftemp.csv"), sep="\t")
colonnes = list(dftemp.columns)
classes = colonnes[1:]
df = pd.DataFrame(test_predictions,columns=classes)
df["text"] = test
df.set_index('text', inplace=True)

fich_result = os.path.join(dossier_resultats,str(fichier_json.split("/")[-1])[:-5]+"_classification_multilabel.csv")
df.to_csv(fich_result, index=True, sep="\t")


# Sauvegarde vers un JSON avec blocs texte annotés
compteur = 0
for textblock_id in dico2["Text_Blocks"] :
  tb_classes = [classes[i] for i, x in enumerate(test_predictions[compteur]) if x == 1]
  try :
    dico2["Text_Blocks"][textblock_id]["Etiquettes"] = dico2["Text_Blocks"][textblock_id]["Etiquettes"]+tb_classes
  except KeyError : 
    dico2["Text_Blocks"][textblock_id]["Etiquettes"] = tb_classes
  compteur+=1

json_result = os.path.join(dossier_resultats,str(fichier_json.split("/")[-1])[:-5]+"_classification_binaire.json")
with open(json_result, "w") as fil:
  json.dump(dico2, fil)
print("Les résultats peuvent être visualisés dans le fichier JSON : ", "/".join(json_result.split("/")[-2:]))

print("Les résultats peuvent être visualisés dans le tableau : ", "/".join(fich_result.split("/")[-2:]))

df.head()
